In [0]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe
import itertools
import pandas as pd

In [0]:
tf.enable_eager_execution()


In [0]:
BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 24 #@param {type:"integer"}

In [0]:
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.conv(inputs)))

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h

In [0]:
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
# model = DavidNet()
# batches_per_epoch = len_train//BATCH_SIZE + 1

# lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
# global_step = tf.train.get_or_create_global_step()
# lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
# opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
# data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
# t = time.time()
# test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

# for epoch in range(EPOCHS):
#   train_loss = test_loss = train_acc = test_acc = 0.0
#   train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

#   tf.keras.backend.set_learning_phase(1)
#   for (x, y) in tqdm(train_set):
#     with tf.GradientTape() as tape:
#       loss, correct = model(x, y)

#     var = model.trainable_variables
#     grads = tape.gradient(loss, var)
#     for g, v in zip(grads, var):
#       g += v * WEIGHT_DECAY * BATCH_SIZE
#     opt.apply_gradients(zip(grads, var), global_step=global_step)

#     train_loss += loss.numpy()
#     train_acc += correct.numpy()

#   tf.keras.backend.set_learning_phase(0)
#   for (x, y) in test_set:
#     loss, correct = model(x, y)
#     test_loss += loss.numpy()
#     test_acc += correct.numpy()
    
#   print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

In [0]:
# lr_range = [0.1, 0.2]
# slanted_triangle_mid_epoch_range = [4,5]
lr_range = [0.6, 0.7, 0.8, 0.9, 1.0]
slanted_triangle_mid_epoch_range = [4,5,6,7,8]
initial_global_step = tf.train.get_or_create_global_step()
ll = []
end_percentage = 0.10

for element in itertools.product(lr_range, slanted_triangle_mid_epoch_range):
#   print(element)
  lr_option = element[0]
  cycle_mid_option = element[1]
  model = DavidNet()
  batches_per_epoch = len_train//BATCH_SIZE + 1
  cycle_mid_epoch = cycle_mid_option
  cycle_end_epoch = int((1. - end_percentage)*EPOCHS)
#   print(cycle_mid_epoch, cycle_end_epoch)
  
#   lr_schedule = np.interp(np.arange(EPOCHS+1), [0, cycle_mid_epoch, cycle_end_epoch, EPOCHS], [lr_option/10.0, lr_option, lr_option/10.0, 0])[0]
#   print(lr_schedule)
#   global_step = tf.train.get_or_create_global_step()
#   lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
#   opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
#   data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

  schedule = np.interp(np.arange(EPOCHS+1), [0, cycle_mid_epoch, cycle_end_epoch, EPOCHS], [lr_option/10.0, lr_option, lr_option/10.0, 0])
  lr_schedule = lambda t: schedule[t]

#   lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
  global_step = initial_global_step.assign(0)
  print(global_step)
  lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
  opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
  data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)
  
  t = time.time()
  test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

  for epoch in range(EPOCHS):
    train_loss = test_loss = train_acc = test_acc = 0.0
    train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

    tf.keras.backend.set_learning_phase(1)
    for (x, y) in tqdm(train_set):
      with tf.GradientTape() as tape:
        loss, correct = model(x, y)

      var = model.trainable_variables
      grads = tape.gradient(loss, var)
      for g, v in zip(grads, var):
        g += v * WEIGHT_DECAY * BATCH_SIZE
      opt.apply_gradients(zip(grads, var), global_step=global_step)

      train_loss += loss.numpy()
      train_acc += correct.numpy()

    tf.keras.backend.set_learning_phase(0)
    for (x, y) in test_set:
      loss, correct = model(x, y)
      test_loss += loss.numpy()
      test_acc += correct.numpy()

    print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)
    ll.append([lr_option, cycle_mid_option, epoch+1, lr_schedule(epoch+1), train_loss / len_train, train_acc / len_train, test_loss / len_test, test_acc / len_test])
    
# ll = pd.concat(ll)
# ll.to_csv('ll.csv', index = False)


<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=0>



epoch: 1 lr: 0.195 train loss: 1.3006213494873047 train acc: 0.53066 val loss: 1.0095890045166016 val acc: 0.6483 time: 45.96309971809387



epoch: 2 lr: 0.33 train loss: 1.024126364135742 train acc: 0.6409 val loss: 0.7795204818725586 val acc: 0.7328 time: 79.54194474220276



epoch: 3 lr: 0.465 train loss: 0.717917857055664 train acc: 0.75442 val loss: 1.2350577880859375 val acc: 0.6397 time: 113.42154884338379



epoch: 4 lr: 0.6 train loss: 0.5977474792480468 train acc: 0.79714 val loss: 0.5664353042602539 val acc: 0.8144 time: 147.356627702713


...


epoch: 6 lr: 0.5228571428571429 train loss: 0.4301788107299805 train acc: 0.8516 val loss: 0.7317071105957031 val acc: 0.7624 time: 206.93022632598877



epoch: 7 lr: 0.6 train loss: 0.37296254684448243 train acc: 0.87282 val loss: 0.4429328796386719 val acc: 0.8502 time: 241.37115573883057



epoch: 8 lr: 0.5614285714285714 train loss: 0.32743356536865237 train acc: 0.88656 val loss: 0.5754843582153321 val acc: 0.8127 time: 275.73080039024353



epoch: 9 lr: 0.5228571428571428 train loss: 0.26608251556396484 train acc: 0.90724 val loss: 0.45368404693603515 val acc: 0.8571 time: 310.14726972579956



epoch: 10 lr: 0.48428571428571426 train loss: 0.22343623977661134 train acc: 0.92242 val loss: 0.32907705841064455 val acc: 0.8933 time: 344.54994678497314


In [0]:
ld = pd.DataFrame(ll, columns = ['max_lr', 'cycle_mid', 'epoch', 'lr', 'train loss', 'train acc', 'val loss', 'val acc'])
ld.to_csv('ll2.csv', index = False)

In [0]:
# lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
# global_step = tf.train.get_or_create_global_step()
# lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE

In [0]:
# # lr_schedule
# lr_func

<function __main__.<lambda>>

In [0]:
# lr_range = [0.1, 0.2]
# slanted_triangle_mid_epoch_range = [4,5]
# initial_global_step = tf.train.get_or_create_global_step()
# for element in itertools.product(lr_range, slanted_triangle_mid_epoch_range):
#   global_step = initial_global_step
#   print(global_step)
  

<tf.Variable 'global_step:0' shape=() dtype=int64, numpy=0>
<tf.Variable 'global_step:0' shape=() dtype=int64, numpy=0>
<tf.Variable 'global_step:0' shape=() dtype=int64, numpy=0>
<tf.Variable 'global_step:0' shape=() dtype=int64, numpy=0>
